<a href="https://colab.research.google.com/github/Wahiba275/Risk_Assessment_Chatbot/blob/main/chatbotV6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=bc08b7271a94cad682d399355e13872a20e5c79e29c26528f1afb051177e6db1
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
from langchain.agents import AgentType,Tool, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper
from langchain.llms import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, Type
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [ ]:
import openai
import getpass
openai.api_key = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
serpapi_api_key= getpass.getpass('SERPAPI API Key:')

SERPAPI API Key:··········


In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import openai
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class RiskInput(BaseModel):
    question: str = Field()

llm = ChatOpenAI(api_key=openai.api_key)
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "you are an AI bot ,think step by step ,starting by checking if the user request is related to risk assessment and denying it if its not related"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)


tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about Risk assessment",
    ),
]


system_message =SystemMessage(content=f"""You are an advanced chatbot designed to answer questions related to risk assessment. Your primary goal is to provide accurate and informative responses to queries about risk assessment. Whenever you receive a user question, quickly decide if it is about risk assessment. If it is, respond with a detailed and relevant answer. If the question is not about risk assessment, politely reply with: 'I'm sorry, I can only answer questions related to risk assessment''""")
agent = initialize_agent(
tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, agent_kwargs ={"system_message":system_message}, system_message =system_message
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm_chain = LLMChain(llm=llm, prompt=chat_template, verbose=True, memory=memory)
tools.append(
    Tool.from_function(
        func=llm_chain.run ,
        name="WebBrowser",
        description="useful for when you need to answer questions about Risk assessment",
        args_schema=RiskInput,

    )
)


class CustomWebBrowserTool(BaseTool):
    name = "WebBrowser"
    description = "useful for when you need to answer questions about risk assessment"
    args_schema: Type[BaseModel] = RiskInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""

        return llm_chain.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("WebBrowser does not support async")

In [ ]:
agent = initialize_agent(
tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True, agent_kwargs ={"system_message":system_message}, system_message =system_message
)

In [ ]:
agent.run("what is a chatbot")



> Entering new AgentExecutor chain...
I'm sorry, I can only answer questions related to risk assessment.

> Finished chain.


"I'm sorry, I can only answer questions related to risk assessment."

In [ ]:
agent.run("c'est quoi l'apprentissage automatique")



> Entering new AgentExecutor chain...
Je suis désolé, mais je ne peux répondre qu'aux questions relatives à l'évaluation des risques.

> Finished chain.


"Je suis désolé, mais je ne peux répondre qu'aux questions relatives à l'évaluation des risques."

In [ ]:
agent.run("what is risk assessment")



> Entering new AgentExecutor chain...
Risk assessment is the process of identifying, evaluating, and analyzing potential risks or hazards that may arise in a particular situation or environment. It involves assessing the likelihood of a risk occurring and the potential consequences or impacts it may have. The purpose of risk assessment is to make informed decisions and take appropriate actions to mitigate or manage the identified risks. It is commonly used in various fields, including but not limited to, occupational health and safety, environmental management, project management, and financial risk analysis.

> Finished chain.


'Risk assessment is the process of identifying, evaluating, and analyzing potential risks or hazards that may arise in a particular situation or environment. It involves assessing the likelihood of a risk occurring and the potential consequences or impacts it may have. The purpose of risk assessment is to make informed decisions and take appropriate actions to mitigate or manage the identified risks. It is commonly used in various fields, including but not limited to, occupational health and safety, environmental management, project management, and financial risk analysis.'

In [ ]:
agent.run("How do you identify and prioritize risks during the risk assessment process?")



> Entering new AgentExecutor chain...
During the risk assessment process, identifying and prioritizing risks is crucial for effective risk management. Here are the steps involved in identifying and prioritizing risks:

1. Risk Identification: This step involves identifying all potential risks that may impact the project, process, or organization. This can be done through various techniques such as brainstorming, checklists, historical data analysis, interviews, and expert judgment.

2. Risk Analysis: Once the risks are identified, they need to be analyzed in terms of their likelihood of occurrence and potential impact. This analysis helps in understanding the significance of each risk and its potential consequences.

3. Risk Evaluation: In this step, the identified risks are evaluated based on their potential impact on the objectives of the project or organization. Risks can be categorized as high, medium, or low based on their severity.

4. Risk Prioritization: After evaluating the 

'During the risk assessment process, identifying and prioritizing risks is crucial for effective risk management. Here are the steps involved in identifying and prioritizing risks:\n\n1. Risk Identification: This step involves identifying all potential risks that may impact the project, process, or organization. This can be done through various techniques such as brainstorming, checklists, historical data analysis, interviews, and expert judgment.\n\n2. Risk Analysis: Once the risks are identified, they need to be analyzed in terms of their likelihood of occurrence and potential impact. This analysis helps in understanding the significance of each risk and its potential consequences.\n\n3. Risk Evaluation: In this step, the identified risks are evaluated based on their potential impact on the objectives of the project or organization. Risks can be categorized as high, medium, or low based on their severity.\n\n4. Risk Prioritization: After evaluating the risks, they need to be prioriti